<a href="https://colab.research.google.com/github/mini1115/TIL/blob/master/%EB%B9%85%EB%B6%84%EA%B8%B0%20%EA%B8%B0%EC%B6%9C/8%ED%9A%8C%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [311]:
from google.colab import files
uploaded = files.upload()

Saving tourist.csv to tourist.csv
Saving piq.csv to piq.csv
Saving chem.csv to chem.csv
Saving churn.csv to churn.csv
Saving churn_test.csv to churn_test.csv
Saving churn_train.csv to churn_train.csv
Saving drinks.csv to drinks.csv


# **작업형1**

Q1.
- 대륙별 맥주 소비량의 평균을 계산하고
- 평균이 가장 큰 대륙을 찾으시오.
- 찾은 대륙에서 맥주 소비량이 6번째 많은 국가의 맥주 소비량을 구하시오.

In [331]:
import pandas as pd
df = pd.read_csv('drinks.csv')

ans = df.groupby('continent')['beer_servings'].mean()
ans.sort_values(ascending=False)
best_continent = 'Europe'

result = df.loc[df['continent']==best_continent]
final = result.sort_values('beer_servings',ascending=False)
final.head(5)

# Ireland
df.loc[df.country=='Ireland'].beer_servings
# 313
final.iloc[4,1]

313

Q2.
- 관광객비율이 두번쨰로 높은 국가의 '사업' 방문객 수를 a라고 정의하시오.
  - 관광객비율 : 관광 / 방문객 합계
  - 방문객 합계 : 관광+공무+사업+기타
- '관광'이 두 번째로 높은 국가의 '공무' 방문객 수를 b라고 정의하시오.
- a와 b의 합을 구하시오.

In [338]:
import pandas as pd
df = pd.read_csv('tourist.csv')
df['방문객 합계'] = df['관광']+df['공무']+df['사업']+df['기타']
df['관광객 비율'] = df['관광'] / df['방문객 합계']
ans = df.sort_values('관광객 비율',ascending=False)
a = ans.iloc[1]['사업']
ans2 = df.sort_values('관광',ascending=False)
b = ans2.iloc[1]['공무']

print(a+b)

441


Q3.
- 주어진 데이터에서 'co' 와 'nmhc' 컬럼을 각각 min Max 스케일링 하시오
- 스케일링된 'co' , 'nmhc' 컬럼의 표준편차를 각각 구하시오.
- 'co' 컬럼의 표준편차에서 'nmhc' 컬럼의 표준편차를 뺀 값을 반올림하시오.

In [347]:
import pandas as pd
df = pd.read_csv('chem.csv')

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['co'] = scaler.fit_transform(df[['co']])
df['nmhc'] = scaler.fit_transform(df[['nmhc']])

co = df['co'].std()
# 0.2856516497116944
nm = df['nmhc'].std()
# 0.3030617020578397
ans = (co-nm)
print(round(ans,3))

-0.017


# **작업형2**

Q. 통신사에서 고객에게 청구될 총 금액을 예측하시오.
  - 예측할 컬럼 TotalCharges(총 청구액)
  - pred : 예측된 총 청구액
  - 제출 파일명 'result.csv'
  - 평가지표 MAE


In [368]:
train = pd.read_csv('churn_train.csv')
test = pd.read_csv('churn_test.csv')

# train.shape,test.shape
# train.info()
# train.describe(include='O')
train.pop('customerID')
test.pop('customerID')
target = train.pop('TotalCharges')

combined = pd.concat([train,test])
combined_oh = pd.get_dummies(combined)
train = combined_oh[:len(train)]
test = combined_oh[len(train):]

# from sklearn.preprocessing import LabelEncoder
# cols = train.select_dtypes(include='O').columns

# for col in cols:
#   le = LabelEncoder()
#   train[col] = le.fit_transform(train[col])
#   test[col] = le.transform(test[col])


from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val = train_test_split(train,target,test_size=0.2,random_state=0)
# x_tr.shape,x_val.shape,y_tr.shape,y_val.shape

# from sklearn.ensemble import RandomForestRegressor
# rf = RandomForestRegressor(random_state=0)
# rf.fit(x_tr,y_tr)
# pred = rf.predict(x_val)
# # 937.29

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(x_tr,y_tr)
pred = lr.predict(x_val)
# 914.6725879047845

# import lightgbm as lgb
# lg = lgb.LGBMRegressor()
# lg.fit(x_tr,y_tr)
# pred = lg.predict(x_val)
# # 946.70

from sklearn.metrics import mean_absolute_error
score = mean_absolute_error(y_val,pred)
print(score)

914.6725879047845


# **작업형3**

Q.1 로지스틱 회귀
- 주어진 데이터에서 로지스틱 회귀분석을 수행해 유의확률이 0.05 이상인 유의하지 않은 독립변수의 개수를 구하시오.

- 유의확률이 0.05 미만인 유의한 변수만을 사용해 다시 로지스틱 회귀 분석을 수행하시오. 이 회귀식의 유의한 회귀계수(상수항 포함)의 합계를 구하시오.

- 문제 1.2 에서 수행한 로지스틱 회귀식에서 'DataUsage' 변수가 5만큼 증가할때, 오즈비를 구하시오.


In [386]:
data = pd.read_csv('churn.csv')

from statsmodels.formula.api import logit

formula = 'Churn ~AccountWeeks+ContractRenewal+DataPlan+DataUsage+CustServCalls+DayMins+DayCalls+MonthlyCharge+OverageFee+RoamMins'

model = logit(formula,data=data).fit()
# model.pvalues > 0.05

new_formula = 'Churn ~ DataUsage+DayMins'

new_model = logit(new_formula,data=data).fit()
# new_model.summary()
sum(new_model.params)

import numpy as np

coef = new_model.params['DataUsage']

round(np.exp(coef*5),3)

Optimization terminated successfully.
         Current function value: 0.393603
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.397599
         Iterations 6


0.428

Q2. 다중 선형 회귀

- 주어진 데이터를 이용해 종속변수(PIQ)와 <BR/>
독립변수(Brain,Height,Weight)로 다중 선형 회귀 분석을 수행하시오.
이때 유의확률(p-value)이 가장 작은 변수의 회귀 계수 값을 구하시오.

- 문제 2-1에 적합한 결정계수 값을 구하시오.

- 뇌 크기 : 90 , 키 : 70, 몸무게 : 150 일때 PIQ를 예측하시오.

In [394]:
df = pd.read_csv('piq.csv')

from statsmodels.formula.api import ols
formula = 'PIQ ~Brain	+Height+	Weight'

model = ols(formula,data=df).fit()
# model.pvalues
# Brain 0.000005
model.params['Brain']
# 2.343130100951171

2.343130100951171

In [398]:
round(model.rsquared,2)

0.37

In [402]:
# - 뇌 크기 : 90 , 키 : 70, 몸무게 : 150 일때
new_data = pd.DataFrame({'Brain':[90],'Height':[70],'Weight':[150]})

result = model.predict(new_data)
round(result[0])

106